In [6]:
import csv
from bs4 import BeautifulSoup
import requests
import os
from tqdm import tqdm
import pandas as pd

In [7]:
from urllib.parse import urlparse

# Функция для получения имени файла из URL
def get_filename_from_url(url):
    parsed_url = urlparse(url)
    return os.path.basename(parsed_url.path)

In [8]:
df = pd.read_csv('flats_unique.csv')
flat_urls = df['link']

In [20]:
START_ID = 3020
END_ID = 3021

dir_images = 'images'

csv_errors = open('download_errors.csv', mode='w', newline='', encoding='utf-8')
writer_err = csv.writer(csv_errors)
writer_err.writerow(['flat_url', 'image_url', 'image_name'])

FILE_OUT_NAME = 'url_images_' + str(START_ID) + '_' + str(END_ID) + '.csv'
csv_file = open(FILE_OUT_NAME, mode='w', newline='', encoding='utf-8')
writer = csv.writer(csv_file)

writer.writerow(['flat_url', 'image_url', 'image_name'])

31

In [21]:
%%time

for url_id in tqdm(range(START_ID, END_ID+1), desc='Flat images parsing'):
    flat_link = flat_urls[url_id]
    flat_req = requests.get(flat_link)
    flat_soup = BeautifulSoup(flat_req.text, "html.parser")

    # Находим div с классом "fotorama"
    fotorama_div = flat_soup.find('div', class_='fotorama')

    # Ищем все ссылки на фото внутри этого блока
    if fotorama_div is None:
        writer_err.writerow([flat_link, None, None])
        continue
    image_tags = fotorama_div.find_all('a', href=True)
    if image_tags is None:
        writer_err.writerow([flat_link, None, None])
        continue

    # Скачиваем все изображения и записываем данные в CSV
    for i, tag in enumerate(image_tags):
        image_url = tag['href']
        response = requests.get(image_url)

        # Проверяем успешность запроса
        if response.status_code == 200:
            # Извлекаем оригинальное имя файла из URL
            filename = get_filename_from_url(image_url)
            filepath = os.path.join('images', filename)

            with open(filepath, 'wb') as f:
                f.write(response.content)

            # Записываем данные в CSV файл
            writer.writerow([flat_link, image_url, filename])
            print(f'Фото {i+1} скачано как {filename} и записано в CSV.')
        else:
            writer_err.writerow([flat_link, image_url, filename])
            print(f'Ошибка при скачивании фото {i+1}.')

    print(f'--------end of flat {url_id} / {END_ID} ---------')


csv_file.close()
csv_errors.close()

print('End of program')

Flat images parsing:  50%|███████████████████████████████▌                               | 1/2 [00:02<00:02,  2.08s/it]

Фото 1 скачано как 9189e5bba3519a77f09f9b43c80e26ed_1200x900.jpg и записано в CSV.
Фото 2 скачано как e619cae39a9b6cf08e78024643d96bcc_1200x900.jpg и записано в CSV.
Фото 3 скачано как 61ab58c3210e70509abd4b371a391f32_1200x900.jpg и записано в CSV.
Фото 4 скачано как 2a58c2d2f74d74347add72bd3a1823d2_1200x900.jpg и записано в CSV.


Flat images parsing: 100%|███████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.88s/it]

Фото 5 скачано как GQYNB4rjTj4 и записано в CSV.
--------end of flat 3021 / 3021 ---------
End of program
CPU times: total: 3.7 s
Wall time: 11.8 s


In [17]:
fotorama_div